Notebook responsável para realizar transformações no dataframe de UASG. Ele está sendo usado apenas para testes, código completos serão enviados para o arquivo de transform.py

Carregamento do arquivo bruto de contratos 👇

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from IPython.display import display
from datetime import datetime

# Configura o caminho para os arquivos brutos
raw_data_dir = Path('..') / 'data' / 'raw'

# Localiza o arquivo de órgãos mais recente (pelo formato de data no nome do arquivo)
uasg_files = list(raw_data_dir.glob('uasg_*.csv'))

if uasg_files:
    # Pega o arquivo mais recente baseado na data de modificação
    latest_uasg_file = max(uasg_files, key=lambda x: x.stat().st_mtime)

    # Verificação (para debug)
    print(f'Arquivo de UASG encontrado: {latest_uasg_file}')
    print(f'O arquivo existe? {latest_uasg_file.exists()}')

    # Carrega o Dataframe
    df = pd.read_csv(latest_uasg_file, encoding='utf-8', low_memory=False)

    # Visualiza dimensões e primeiras linhas
    print(f'\nDataframe carregado: {df.shape[0]} linhas, {df.shape[1]} colunas')
    print(f'Primeira 5 colunas: {', '.join(df.columns[:5])}')

else:
    print('ERRO: Nenhum arquivo de contratos encontrado em data/raw')
    print(f'Diretório verificado: {raw_data_dir.resolver()}')
    print(f'O diretório existe? {raw_data_dir.exists()}')
    print('Arquivos no diretório:', list(raw_data_dir.glob('*')))

Arquivo de UASG encontrado: ..\data\raw\uasg_2025-08-03.csv
O arquivo existe? True

Dataframe carregado: 21084 linhas, 22 colunas
Primeira 5 colunas: codigoUasg, nomeUasg, usoSisg, adesaoSiasg, siglaUf


### ANÁLISE GERAL DO DATAFRAME

In [3]:
df.head(10)

,codigoUasg,nomeUasg,usoSisg,adesaoSiasg,siglaUf,codigoMunicipio,codigoMunicipioIbge,nomeMunicipioIbge,codigoUnidadePolo,nomeUnidadePolo,...,uasgCadastradora,cnpjCpfUasg,codigoOrgao,cnpjCpfOrgao,cnpjCpfOrgaoVinculado,cnpjCpfOrgaoSuperior,codigoSiorg,statusUasg,dataImplantacaoSidec,dataHoraMovimento
0,996612,CONSELHO REGIONAL DE CONTABILIDADE - PA,False,True,PA,4278.0,1501402.0,BELÉM,0,NaN,...,False,NaN,38511,4977518000130,3.361857e+13,4.898280e+11,NaN,True,2016-08-24T03:00:00.000+00:00,2023-03-24T15:26:00
1,990224,ESP-AG. DE BIBL. E COLEÇÕES DIGITAIS - USP,False,True,SP,71072.0,3550308.0,SÃO PAULO,990000,ESP-SECRETARIA DE GESTAO E GOVERNO DIGITAL,...,False,NaN,95596,63025530000104,4.637940e+13,0.000000e+00,NaN,True,NaN,2025-06-23T14:40:00
2,990223,ESP-CENTRAL DE COMPRAS - TECNOLÓGICAS,False,True,SP,62910.0,3509502.0,CAMPINAS,990000,ESP-SECRETARIA DE GESTAO E GOVERNO DIGITAL,...,False,NaN,95554,46068425000133,8.678541e+12,4.637940e+13,NaN,True,NaN,2024-03-02T14:46:00
3,990222,ESP-CENTRAL DE COMPRAS - HUMANIDADES E ARTES,False,True,SP,62910.0,3509502.0,CAMPINAS,990000,ESP-SECRETARIA DE GESTAO E GOVERNO DIGITAL,...,False,NaN,95554,46068425000133,8.678541e+12,4.637940e+13,NaN,True,NaN,2024-03-02T14:46:00
4,990221,ESP-CENTRAL DE COMPRAS - EXATAS,False,True,SP,62910.0,3509502.0,CAMPINAS,990000,ESP-SECRETARIA DE GESTAO E GOVERNO DIGITAL,...,False,NaN,95554,46068425000133,8.678541e+12,4.637940e+13,NaN,True,NaN,2024-03-02T14:46:00
5,990220,ESP-CENTRAL DE COMPRAS - BIOLÓGICAS,False,True,SP,62910.0,3509502.0,CAMPINAS,990000,ESP-SECRETARIA DE GESTAO E GOVERNO DIGITAL,...,False,NaN,95554,46068425000133,8.678541e+12,4.637940e+13,NaN,True,NaN,2024-03-02T14:46:00
6,990219,ESP-CENTRAL DE COMPRAS - REITORIA,False,True,SP,62910.0,3509502.0,CAMPINAS,990000,ESP-SECRETARIA DE GESTAO E GOVERNO DIGITAL,...,False,NaN,95554,46068425000133,8.678541e+12,4.637940e+13,NaN,True,NaN,2024-03-02T14:46:00
7,990218,ESP-CENTRAL DE COMPRAS - ÁREA DA SAÚDE,False,True,SP,62910.0,3509502.0,CAMPINAS,990000,ESP-SECRETARIA DE GESTAO E GOVERNO DIGITAL,...,False,NaN,95554,46068425000133,8.678541e+12,4.637940e+13,NaN,True,NaN,2024-03-02T14:46:00
8,990217,ESP-COORDENADORIA DE ENSINO TÉCNICO SUPERIOR,False,True,SP,71072.0,3550308.0,SÃO PAULO,990000,ESP-SECRETARIA DE GESTAO E GOVERNO DIGITAL,...,False,NaN,98501,8678541000185,4.637940e+13,0.000000e+00,NaN,True,NaN,2025-08-01T16:26:00
9,990216,"ESP-DIRETORIA DE CIÊNCIA,TECNOLOGIA INOVAÇÃO",False,True,SP,71072.0,3550308.0,SÃO PAULO,990000,ESP-SECRETARIA DE GESTAO E GOVERNO DIGITAL,...,False,NaN,98501,8678541000185,4.637940e+13,0.000000e+00,NaN,True,NaN,2025-08-01T16:24:00


Verificação de Nulos por quantidade e porcentagem 👇

In [2]:
# Calcula contagem e percentual de nulos em cada coluna
nulos = df.isnull().sum()
percentual = (df.isnull().sum() / len(df) * 100).round(2)

# Cria um Dataframe simples apenas com colunas que têm pelo menos um valor nulo
colunas_com_nulos = pd.DataFrame({
    'Valores Nulos': nulos[nulos >0],
    'Percentual (%)': percentual[nulos > 0]
}).sort_values('Percentual (%)', ascending=False)

# Exibe o resultado
colunas_com_nulos

,Valores Nulos,Percentual (%)
nomeUnidadeEspelho,20072,95.20
codigoSiorg,17155,81.37
nomeUnidadePolo,15454,73.30
cnpjCpfUasg,12766,60.55
dataImplantacaoSidec,3750,17.79
cnpjCpfOrgaoSuperior,2014,9.55
cnpjCpfOrgaoVinculado,1823,8.65
codigoMunicipioIbge,242,1.15
nomeMunicipioIbge,242,1.15
codigoMunicipio,22,0.10


### ANÁLISE DAS COLUNAS NULAS